In [ ]:
#importing all the stuff

import numpy as np
import math as math
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox
pd.options.display.max_rows = 100


def lower_case_column_names(df):
    df.columns=[i.lower() for i in df.columns]
    return df

def clean_state_column(dataframe):
    dataframe.rename(columns={'st':'state'}, inplace=True)
    return dataframe

In [ ]:
#lowercasing and indexing the columns of the three different files

file1 = pd.read_csv('file1.csv')
file1 = lower_case_column_names(file1)
file1 = file1 = file1.sort_index(axis=1)
clean_state_column(file1)

In [ ]:
file2 = pd.read_csv('file2.csv')
file2 = lower_case_column_names(file2)
file2 = file2 = file2.sort_index(axis=1)
clean_state_column(file2)

In [ ]:
file3 = pd.read_csv('file3.csv')
file3 = lower_case_column_names(file3)
file3 = file3 = file3.sort_index(axis=1)
clean_state_column(file3)

In [ ]:
#ensuring all have the same format and merging
file1.columns == file2.columns
file2.columns == file3.columns

In [ ]:
df = pd.concat([file1,file2,file3], axis=0)
df


In [ ]:

#drop duplicatees and unnecessary columns
df = df.drop_duplicates()
df = df.drop("customer", axis=1)


In [ ]:
df.info()

In [ ]:
#cleaning some data


df['customer lifetime value'] = (
    df['customer lifetime value'].apply(lambda x: float(x.replace('%', ''))/100 if type(x) == str else x))
#convert Customer Lifetime Value into integer by first converting it into float
df['customer lifetime value'] = pd.to_numeric(df["customer lifetime value"], errors = 'coerce')
df['customer lifetime value'] = df['customer lifetime value'].apply(lambda x: math.trunc(x) if np.isnan(x) != True else 0)

df.info()

In [ ]:
#cleaning all the states
df["state"].unique()


In [ ]:
def clean_state(x):
    if x in ['AZ', 'Arizona']:
        return 'Arizona'
    elif x in ['Cali', 'California']:
        return 'California'
    elif x in ['Washington', 'WA']:
        return 'Washington'
    elif x in ['Nevada']:
        return 'Nevada'
    elif x in ['Oregon']:
        return 'Oregon'
        

In [ ]:
df['state'] = list(map(clean_state, df['state'])) 

In [ ]:
df['state'].unique()

In [ ]:
#cleaning the gender column
df["gender"].unique()

In [ ]:
def clean_gender(x):
    if x in ['M', 'Male']:
        return 'M'
    elif x in ['F', 'Femal', 'female']:
        return 'F'
    else:
        return 'NaN'

In [ ]:
df['gender'] = list(map(clean_gender, df['gender'])) 
df['gender'].value_counts()

In [ ]:
#cleaning the number of open complaints by extracting the middle number



df['number of open complaints'] = df['number of open complaints'].astype(str).str.replace('(.*)/0/(.*)','0', regex=True)


In [ ]:
df['number of open complaints'] = df['number of open complaints'].astype(str).str.replace('(.*)/1/(.*)','1', regex=True)


In [ ]:
df['number of open complaints'] = df['number of open complaints'].astype(str).str.replace('(.*)/2/(.*)','2', regex=True)


In [ ]:
df['number of open complaints'] = df['number of open complaints'].astype(str).str.replace('(.*)/3/(.*)','3', regex=True)


In [ ]:
df['number of open complaints'] = df['number of open complaints'].astype(str).str.replace('(.*)/4/(.*)','4', regex=True)


In [ ]:
df['number of open complaints'] = df['number of open complaints'].astype(str).str.replace('(.*)/5/(.*)','5', regex=True)

In [ ]:
df['number of open complaints'] =  pd.to_numeric(df['number of open complaints'], errors='coerce')

In [ ]:
df.info()
#df.reset_index()

In [ ]:
df['number of open complaints'].value_counts()

In [ ]:
#cleaning the income column, replacing 0 with NaN, replacing NaN with the mean

df['income'] = df['income'].replace(0, np.nan)
df

In [ ]:
df['total claim amount'] = pd.to_numeric(df['total claim amount'], errors='coerce')
mean_tca = df['total claim amount'].mean()
mean_tca
df['income'] = pd.to_numeric(df['income'], errors='coerce')
#mean_i = df['income'].mean()
#mean_i

In [ ]:
#replacing the missing values of total claim amount with the mean of tca


df['total claim amount'] = df['total claim amount'].fillna(mean_tca)
df['total claim amount'].value_counts()
df.info()

In [ ]:
#replacing the missing values of income with the mean of income


df['income'] = df['income'].fillna(df['income'].mean())
df['income'].value_counts()

In [ ]:
#replacing the missing values of monthly premium auto with the mean of monthly premium auto

df['monthly premium auto'] = df['monthly premium auto'].fillna(df['monthly premium auto'].mean())
df['monthly premium auto']

In [ ]:
#replacing the missing values of customer lifetime value
df['customer lifetime value'] = df['customer lifetime value'].fillna(df['customer lifetime value'].mean())
df['customer lifetime value']

In [ ]:
#converting state into regions via function
df = df.rename(columns={'state': 'region'})

In [ ]:
def state_region(x):
    if x in ['California']:
        return 'West'
    elif x in ['Oregon']:
        return 'Northwest'
    elif x in ['Washington']:
        return 'East'
    elif x in ['Arizona', 'Nevada']:
        return 'Central'

#state_dict = {}

In [ ]:
df['region'] = list(map(state_region, df['region']))
df.head()


In [ ]:
#clean the vehicle class by merging the luxury classes
df['vehicle class'].unique()

In [ ]:
#merge the ctaegories
merge_dict = {'Luxury SUV': 'Luxury Vehicle', 'Luxury Car': 'Luxury Vehicle'}
df['vehicle class'] = df['vehicle class'].replace(merge_dict)

In [ ]:
#df['vehicle class'].unique()
df.head(100)

In [ ]:
#removing the outliers with the help of the zscore
from scipy.stats import zscore

In [ ]:


z = np.abs(zscore(df['monthly premium auto']))
df = df[(z < 2.698)]


In [ ]:
#y = np.abs(zscore(df['total claim amount']))
#df = df[(y < 2.698)]

In [ ]:
x = np.abs(zscore(df['income']))
df = df[(x < 2.698)]

In [ ]:
df.info()

In [ ]:
#rounding the numbers
df['income'] = df['income'].round()
df['total claim amount'] = df['total claim amount'].round()
df['customer lifetime value'] = df['customer lifetime value'].round(1)

In [ ]:
#standardizing the text to lower case

df['education'] = df['education'].str.lower()
df['gender'] = df['gender'].str.lower()
df['policy type'] = df['policy type'].str.lower()
df['region'] = df['region'].str.lower()
df['vehicle class'] = df['vehicle class'].str.lower()

In [ ]:
df.head()